In [0]:
%pip install faker

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col, expr
from pyspark.sql.types import StringType, DateType
from faker import Faker
import pandas as pd

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("GenerateFakeData") \
    .config("spark.sql.shuffle.partitions", "500") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "16") \
    .getOrCreate()

In [0]:

fake = Faker()

# UDFs for generating fake data
def generate_first_name():
    return fake.first_name()

def generate_last_name():
    return fake.last_name()

def generate_address():
    return fake.address()

def generate_dob():
    return fake.date_of_birth(minimum_age=18, maximum_age=90).strftime("%Y-%m-%d")

first_name_udf = udf(generate_first_name, StringType())
last_name_udf = udf(generate_last_name, StringType())
address_udf = udf(generate_address, StringType())
dob_udf = udf(generate_dob, StringType())

In [0]:
first_name_udf = udf(generate_first_name, StringType())
last_name_udf = udf(generate_last_name, StringType())
address_udf = udf(generate_address, StringType())
dob_udf = udf(generate_dob, StringType())

# Create a DataFrame with the required number of rows
num_records = 30000000  # Adjust based on the size needed
df = spark.range(num_records) \
    .select(
        first_name_udf().alias("first_name"),
        last_name_udf().alias("last_name"),
        address_udf().alias("address"),
        dob_udf().alias("dob")
    )



In [0]:
# Repartition to balance the load
df = df.repartition(300)  # Increase partitions if necessary

# Write the DataFrame to CSV
output_path = "dbfs:/dbfs/Workspace/Users/kratikamahale_outlook.com#ext#@kratikamahaleoutlook.onmicrosoft.com/databricks_training/output3.csv"  # Use DBFS path for Databricks
df.write.csv(output_path, header=True, mode="overwrite")

print(f"Data saved at: {output_path}")


Data saved at: dbfs:/dbfs/Workspace/Users/kratikamahale_outlook.com#ext#@kratikamahaleoutlook.onmicrosoft.com/databricks_training/output3.csv


In [0]:
# Path to the CSV file in DBFS


# Read the CSV file from DBFS
df = spark.read.csv(output_path, header=True, inferSchema=True)

# Define the masking functions
def mask_name(name):
    return name[0] + "*" * (len(name) - 1)  # Mask all but the first character

def mask_address(address):
    return "Address Masked"

# Register UDFs for masking
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

mask_name_udf = udf(mask_name, StringType())
mask_address_udf = udf(mask_address, StringType())

# Apply masking to the DataFrame
masked_df = df.withColumn("first_name", mask_name_udf(col("first_name"))) \
              .withColumn("last_name", mask_name_udf(col("last_name"))) \
              .withColumn("address", mask_address_udf(col("address")))
masked_df.show(50)


+------------------+----------+--------------+----+
|        first_name| last_name|       address| dob|
+------------------+----------+--------------+----+
|             K****|   R******|Address Masked|NULL|
|       A**********| *********|Address Masked|NULL|
|         K********|   S******|Address Masked|NULL|
|         W********| *********|Address Masked|NULL|
|           M******|    D*****|Address Masked|NULL|
|        A*********| *********|Address Masked|NULL|
|             A****|     S****|Address Masked|NULL|
|   N**************| *********|Address Masked|NULL|
|             R****|    G*****|Address Masked|NULL|
|  L***************| *********|Address Masked|NULL|
|              J***|     A****|Address Masked|NULL|
|    P*************| *********|Address Masked|NULL|
|            A*****|    P*****|Address Masked|NULL|
|      N***********| *********|Address Masked|NULL|
|             K****|   R******|Address Masked|NULL|
|     C************| *********|Address Masked|NULL|
|          J

In [0]:
output_path = "dbfs:/dbfs/Workspace/Users/kratikamahale_outlook.com#ext#@kratikamahaleoutlook.onmicrosoft.com/databricks_training/masked.csv"  # Use DBFS path for Databricks
masked_df.write.csv(output_path, header=True, mode="overwrite")

In [0]:
# List the files in a directory and their sizes
dbutils.fs.ls(output_path)


[FileInfo(path='dbfs:/dbfs/Workspace/Users/kratikamahale_outlook.com#ext#@kratikamahaleoutlook.onmicrosoft.com/databricks_training/masked.csv/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1725612487000),
 FileInfo(path='dbfs:/dbfs/Workspace/Users/kratikamahale_outlook.com#ext#@kratikamahaleoutlook.onmicrosoft.com/databricks_training/masked.csv/_committed_3832948855196801319', name='_committed_3832948855196801319', size=2249, modificationTime=1725612487000),
 FileInfo(path='dbfs:/dbfs/Workspace/Users/kratikamahale_outlook.com#ext#@kratikamahaleoutlook.onmicrosoft.com/databricks_training/masked.csv/_started_3832948855196801319', name='_started_3832948855196801319', size=0, modificationTime=1725611215000),
 FileInfo(path='dbfs:/dbfs/Workspace/Users/kratikamahale_outlook.com#ext#@kratikamahaleoutlook.onmicrosoft.com/databricks_training/masked.csv/part-00000-tid-3832948855196801319-148ed0ab-0ef6-428f-83b0-934c207caea4-731-1-c000.csv', name='part-00000-tid-3832948855196801319-148ed0ab

In [0]:
df2 = spark.read.csv(output_path, header=True, inferSchema=True)
df2 = df2.coalesce(1)

In [0]:
# Write to local file system (on the driver)
df2.write.csv("file:/tmp/local_csv_files_final/", header=True)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1984032826532986>, line 2
      1 # Write to local file system (on the driver)
----> 2 df2.write.csv("file:/tmp/local_csv_files_final/", header=True)

NameError: name 'df2' is not defined

In [0]:
import os

# Specify the source and destination paths
source_path = "/tmp/local_csv_files_final/"
destination_path = "dbfs:/tmp/local_csv_files_final/"

# Check if the source file exists
if os.path.exists(source_path):
    # Copy the files from the local file system to DBFS
    dbutils.fs.cp("file:" + source_path, destination_path, recurse=True)
    print("Files copied successfully!")
else:
    print("Source file does not exist.")

Source file does not exist.
